# 第16章 深度学习初窥之神经网络模型 - by 王宇韬

# 1.神经网络模型简单代码实现

神经网络分类模型：MLPClassifier

In [1]:
X = [[1, 0], [5, 1], [6, 4], [4, 2], [3, 2]]
y = [0, 1, 1, 0, 0]

In [2]:
from sklearn.neural_network import MLPClassifier
mlp =MLPClassifier()
mlp.fit(X, y)

D:\Anaconda\Anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [3]:
y_pred = mlp.predict(X)

In [4]:
import pandas as pd
a = pd.DataFrame()  # 创建一个空DataFrame 
a['预测值'] = list(y_pred)
a['实际值'] = list(y)

In [5]:
a

,预测值,实际值
0,0,0
1,1,1
2,1,1
3,0,0
4,0,0


补充知识点 - 神经网络回归模型：MLPRegressor

In [7]:
from sklearn.neural_network import MLPRegressor
X = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
y = [1, 2, 3, 4, 5]

model = MLPRegressor(random_state=123)  # 设置random_state随机状态参数，使得每次训练的模型都是一样的
model.fit(X, y)

print(model.predict([[5, 5]]))

[2.85598566]


# 2.案例实战 - 用户评论情感分析

# 2.1 数据读取、中文分词、文本向量化

1.数据读取

In [1]:
import pandas as pd
df = pd.read_excel('产品评价.xlsx')
df.head()

,客户编号,评论,评价
0,1,是iPhone8 XR正品，按键屏幕反应蛮快的很灵活，屏幕6.0的不算很大，刚刚好，这款面容...,1
1,2,外形外观：外光非常漂亮，黑色的非常大气。适合男士拥有。屏幕音效：刚开机就下载了一个QQ音乐试...,1
2,3,从苹果4s，到6s，再到xr，就是喜欢苹果的手感和风格，视频流畅，图片清晰，纠结了好久买哪个...,1
3,4,主要是手感，太沉了，比苹果6，沉一倍，厚太多了，看中双卡双待机，刚买回来用，待机时间还不错，...,1
4,5,外形外观：红色超级好看，送妈妈的。屏幕音效：音效还可以，也什么特别的，屏幕看着也挺舒服。拍照...,1


2.中文分词

In [2]:
# jieba库分词示例
import jieba
word = jieba.cut('我爱北京天安门')
for i in word:
    print(i)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\85079\AppData\Local\Temp\jieba.cache
Loading model cost 1.143 seconds.
Prefix dict has been built succesfully.


我
爱
北京
天安门


In [3]:
# 通过第2章讲的iloc获取数据表DataFrame第一行信息，0表示第一行
df.iloc[0]

客户编号                                                    1
评论      是iPhone8 XR正品，按键屏幕反应蛮快的很灵活，屏幕6.0的不算很大，刚刚好，这款面容...
评价                                                      1
Name: 0, dtype: object

In [4]:
# 为了循序渐进，这里先演示第一条评论的分词效果
import jieba
word = jieba.cut(df.iloc[0]['评论'])
result = ' '.join(word)
print(result)

是 iPhone8   XR 正品 ， 按键 屏幕 反应 蛮快 的 很 灵活 ， 屏幕 6.0 的 不算 很大 ， 刚刚 好 ， 这 款 面容 识别 开锁 比 指纹 方便 多 了 ， 内外 的 整体 看起来 很 美观 ， 整机 子 不算 是 很厚感 ， 像素 高 比较 清晰 ， 双卡 双待 ， 续航 强 ， 跟 8plus 差价 300 元 ， 还是 选 XR 款好 ， 性能 不错 ， 处理器 、 芯片 也 是 最新 一代


In [5]:
# 遍历整张表格，对所有评论进行分词
words = []
for i, row in df.iterrows():
    word = jieba.cut(row['评论'])
    result = ' '.join(word) 
    words.append(result)

In [6]:
words[0:3]

['是 iPhone8   XR 正品 ， 按键 屏幕 反应 蛮快 的 很 灵活 ， 屏幕 6.0 的 不算 很大 ， 刚刚 好 ， 这 款 面容 识别 开锁 比 指纹 方便 多 了 ， 内外 的 整体 看起来 很 美观 ， 整机 子 不算 是 很厚感 ， 像素 高 比较 清晰 ， 双卡 双待 ， 续航 强 ， 跟 8plus 差价 300 元 ， 还是 选 XR 款好 ， 性能 不错 ， 处理器 、 芯片 也 是 最新 一代',
 '外形 外观 ： 外光 非常 漂亮 ， 黑色 的 非常 大气 。 适合 男士 拥有 。 屏幕 音效 ： 刚 开机 就 下载 了 一个 QQ 音乐 试 了 一下 。   音效 还是 非常 不错 的 。 拍照 效果 ： 拍照 很 清晰 ， 照亮 你 脸上 的 痘痘 。 运行 速度 ： 运行 速度 就 不用说 了 。   一个 字快 。 待机时间 ： 待机 很 不错 。 用 一段时间 再 来 评价 。 其他 特色 ： 个人感觉 比 Ｘ 好 。   可能 是因为 上手 的 手感 比较 好 吧 ， 总之 还是 值得 入手 的',
 '从 苹果 4s ， 到 6s ， 再 到 xr ， 就是 喜欢 苹果 的 手感 和 风格 ， 视频 流畅 ， 图片 清晰 ， 纠结 了 好久 买 哪个 颜色 ， 白色 干净 ， 同事 买 的 黄色 ， 感觉 也 很 好看 ， 蓝色 ， 珊瑚 我 也 喜欢 ， 最终 还是 选择 比较 适合 女生 的 珊瑚 色 ， 实物 比 图片 更 漂亮 ， 超级 喜欢 ， 运行 速度 快 ， 全屏 显示 ， 体积小 了 ， 可 显示 区域 变得 了 ， 很棒 。']

In [7]:
# 如果对上面过程如果熟悉后，也可以直接写成如下的合并代码形式
words = []
for i, row in df.iterrows():
    words.append(' '.join(jieba.cut(row['评论'])))

In [8]:
# # iterrows()函数相关知识点，不熟悉DataFrame数据表遍历的话，可以把下面的注释取消了，看看效果
# for i, row in df.iterrows():
#     print(i)
#     print(row)

3.文本向量化

In [9]:
# 文本向量化CountVectorizer()函数的使用技巧：使用示例
from sklearn.feature_extraction.text import CountVectorizer
test = ['手机 外观 漂亮', '手机 图片 清晰']
vect = CountVectorizer()
X = vect.fit_transform(test)
X = X.toarray()

In [10]:
words_bag = vect.vocabulary_
print(words_bag)

{'手机': 2, '外观': 1, '漂亮': 4, '图片': 0, '清晰': 3}


In [11]:
# 实际应用
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X = vect.fit_transform(words)
X = X.toarray()
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
words_bag = vect.vocabulary_
print(words_bag)

{'iphone8': 194, 'xr': 264, '正品': 2660, '按键': 2221, '屏幕': 1798, '反应': 1210, '蛮快': 3492, '灵活': 2843, '不算': 517, '很大': 1967, '刚刚': 1031, '面容': 3979, '识别': 3570, '开锁': 1915, '指纹': 2218, '方便': 2362, '内外': 941, '整体': 2341, '看起来': 3101, '美观': 3345, '整机': 2344, '很厚感': 1959, '像素': 862, '比较': 2704, '清晰': 2808, '双卡': 1201, '双待': 1203, '续航': 3301, '8plus': 143, '差价': 1823, '300': 50, '还是': 3758, '款好': 2655, '性能': 2040, '不错': 538, '处理器': 1460, '芯片': 3455, '最新': 2506, '一代': 290, '外形': 1471, '外观': 1473, '外光': 1468, '非常': 3972, '漂亮': 2832, '黑色': 4068, '大气': 1521, '适合': 3827, '男士': 2997, '拥有': 2207, '音效': 3985, '开机': 1910, '下载': 445, '一个': 280, 'qq': 234, '音乐': 3983, '一下': 276, '拍照': 2203, '效果': 2330, '照亮': 2863, '脸上': 3409, '痘痘': 3019, '运行': 3744, '速度': 3854, '不用说': 514, '字快': 1666, '待机时间': 1951, '待机': 1950, '一段时间': 354, '评价': 3566, '其他': 928, '特色': 2908, '个人感觉': 583, '可能': 1271, '是因为': 2449, '上手': 420, '手感': 2136, '总之': 2041, '值得': 841, '入手': 892, '苹果': 3464, '4s': 81, '6s': 127, '就是': 1782, '喜欢': 1

In [13]:
len(words_bag)

4075

In [27]:
import pandas as pd
# pd.set_option('display.max_columns', None)  # 添加这行代码可以显示所有列，如果讲None改成500，则表示可最多显示500列
# pd.set_option('display.max_rows', None)  # 添加这行代码可以显示所有行，如果讲None改成500，则表示可最多显示500行
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,...,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


4.目标变量提取

In [15]:
y = df['评价']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: 评价, dtype: int64

# 2.2 神经网络模型的搭建与使用

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [28]:
from sklearn.neural_network import MLPClassifier
mlp =MLPClassifier()  # 因为模型运行具有随机性，如果想让每次运行结果一致，可以设置random_state随机参数为任一数字，如MLPClassifier(random_state=123)
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
y_pred = mlp.predict(X_test)
print(y_pred)  # 因为模型运行具有随机性，所以这里得到的结果可能和书上的略有不同，如果想让每次运行结果一致，可以设置random_state随机参数为任一数字，如MLPClassifier(random_state=123)

[1 0 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0
 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1
 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1]


In [30]:
a = pd.DataFrame()  # 创建一个空DataFrame 
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,1,1
1,0,0
2,0,1
3,1,1
4,0,0


In [20]:
# 获取预测准确度
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
score

0.9814814814814815

In [26]:
# 通过模型自带的score()函数也可以获取预测准确度
mlp.score(X_test, y_test)

0.9814814814814815

In [21]:
# 自我体验
comment = input('请输入您对本商品的评价：')
comment = [' '.join(jieba.cut(comment))]
print(comment)
X_try = vect.transform(comment)
y_pred = mlp.predict(X_try.toarray())
print(y_pred)

请输入您对本商品的评价：物流为什么这么慢，到手里都已经一周了。五星给手机，一星给物流。
['物流 为什么 这么 慢 ， 到 手里 都 已经 一周 了 。 五星 给 手机 ， 一星 给 物流 。']
[0]


In [23]:
# 朴素贝叶斯模型对比
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(X_train,y_train)

y_pred = nb_clf.predict(X_test)
print(y_pred)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print(score)

[1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1]
0.8703703703703703
